In [1]:
# File: Notebook_WebData_AccountingData_Combi.ipynb
# Created with assistance of ChatGPT (OpenAI) – reviewed on 2025-05-04
# Author: Maria Heinrich

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 3, Finished, Available, Finished)

In [2]:
import time

start_time = time.time()

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 4, Finished, Available, Finished)

# Exchange rate table

In [14]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from io import StringIO

# Fetch data from the URL
url = "https://www.oenb.at/oearb/zinssatzwechselkurse/download-zeitreihe?start=2023-01-01&end=2023-06-30&codes=AUD,BGN,BRL,CAD,CHF,CNY,CZK,DKK,GBP,HKD,HUF,IDR,ILS,INR,ISK&format=CSV"
response = requests.get(url)

# Read the CSV data
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data, delimiter=';')

# Parse the date string from the first row
date_str = df.at[0, 'Datum']
date_format = '%Y-%m-%d'  # Changed date format
datum = datetime.strptime(date_str, date_format)

# Subtract one day from the date
new_date = datum - timedelta(days=1)

# Replace the date in the first row with the new date
df.at[0, 'Datum'] = new_date.strftime(date_format)

# Melt the DataFrame to create the desired table structure
melted_df = df.melt(id_vars=["Datum"], var_name="CurrencyCode", value_name="ExchangeRate")
melted_df['Datum'] = pd.to_datetime(melted_df['Datum'], format=date_format)

# Fill missing values for each currency separately
melted_df['ExchangeRate'] = melted_df.groupby('CurrencyCode')['ExchangeRate'].ffill()

melted_df.rename(columns={"Datum": "Date"}, inplace=True)

# Define the start and end dates
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 6, 30)

# Generate a list of dates in the range
date_list = pd.date_range(start_date, end_date)

# Create a DataFrame with all combinations of dates and currency codes
date_df = pd.DataFrame({'Date': date_list})
currency_codes = melted_df['CurrencyCode'].unique()
cross_join_df = pd.DataFrame([(date, code) for date in date_list for code in currency_codes], columns=['Date', 'CurrencyCode'])

# Merge the actual data with the cross join DataFrame to ensure all combinations exist
merged_df = pd.merge(cross_join_df, melted_df, on=['Date', 'CurrencyCode'], how='left')

# Fill missing exchange rates by carrying forward the last available exchange rate for each currency
merged_df['ExchangeRate'] = merged_df.groupby('CurrencyCode')['ExchangeRate'].ffill()

# Sort the result DataFrame by 'Datum'
merged_df.sort_values(by='Date', inplace=True)

# Reset index
merged_df.reset_index(drop=True, inplace=True)

# Change datetime to date
merged_df["Date"] = pd.to_datetime(merged_df["Date"]).dt.date

# Convert 'ExchangeRate' to float
merged_df['ExchangeRate'] = merged_df['ExchangeRate'].str.replace('.', '').str.replace(',', '.').astype(float)

# Save the result to CSV
merged_df.to_csv('output_result.csv', index=False)

print(df)
# Save the DataFrame to a CSV file
df.to_csv('FX_Data.csv', index=False)
spark.createDataFrame(df).write.mode("overwrite").option("header", True).csv(f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/Files/FX_Data.csv")


StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 16, Finished, Available, Finished)

          Datum     AUD     BGN     BRL     CAD     CHF     CNY     CZK  \
0    2023-01-01  1,5699  1,9558  5,7048  1,4506  0,9873  7,3698  24,176   
1    2023-01-03  1,5708  1,9558  5,6656  1,4414  0,9879  7,2863  24,124   
2    2023-01-04  1,5452  1,9558  5,7758  1,4363  0,9843  7,2985  24,028   
3    2023-01-05  1,5515  1,9558   5,706  1,4316  0,9839  7,2863  24,028   
4    2023-01-06   1,559  1,9558  5,6034  1,4331  0,9864  7,2045  24,054   
..          ...     ...     ...     ...     ...     ...     ...     ...   
122  2023-06-26  1,6346  1,9558  5,2177  1,4354  0,9737  7,8972  23,649   
123  2023-06-27  1,6369  1,9558  5,2311  1,4408  0,9789  7,8976  23,609   
124  2023-06-28  1,6525  1,9558   5,294  1,4491  0,9822  7,9262  23,683   
125  2023-06-29   1,648  1,9558  5,3006  1,4503  0,9783   7,914  23,686   
126  2023-06-30  1,6398  1,9558  5,2788  1,4415  0,9788  7,8983  23,742   

        DKK      GBP     HKD     HUF        IDR     ILS      INR    ISK  
0    7,4374   0,8863  8,3

In [4]:
# Import required libraries
import pandas as pd
from pyspark.sql import SparkSession
from notebookutils import mssparkutils

# Create a Spark session
spark = SparkSession.builder.getOrCreate()

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(merged_df)

# Define OneLake path
output_path = "Files/output_result.csv"

# Save as CSV to OneLake (Delta Table format is recommended, but CSV works)
spark_df.write.mode("overwrite").option("header", True).csv(f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/{output_path}")

print("CSV saved to OneLake successfully.")


StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 6, Finished, Available, Finished)

CSV saved to OneLake successfully.


In [5]:
merged_df

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 7, Finished, Available, Finished)

,Date,CurrencyCode,ExchangeRate
0,2023-01-01,AUD,1.5699
1,2023-01-01,ISK,151.7000
2,2023-01-01,INR,88.4260
3,2023-01-01,ILS,3.7537
4,2023-01-01,IDR,16631.7800
...,...,...,...
2710,2023-06-30,BGN,1.9558
2711,2023-06-30,AUD,1.6398
2712,2023-06-30,INR,89.2065
2713,2023-06-30,CZK,23.7420


# Fake accounting data

In [6]:
!pip install Faker

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 8, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:0000:0100:01


In [7]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from io import StringIO
import numpy as np
from faker import Faker
from pyspark.sql import SparkSession
from notebookutils import mssparkutils

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 9, Finished, Available, Finished)

In [8]:
# Initialize Faker object
faker = Faker()

# Define the start and end dates
start_date = pd.to_datetime('2023-01-01').timestamp()
end_date = pd.to_datetime('2023-06-30').timestamp()

# Generate 500,000 random dates between the start and end dates
random_dates = pd.to_datetime(np.random.randint(start_date, end_date, size=500000), unit='s').date

# Generate 500,000 random figures between 0.01 and 150,000 and limit to 2 decimal places
random_figures = np.round(np.random.uniform(0.01, 150000, size=500000), 2)

# Generate 500,000 random text strings limited to 50 characters
random_texts = [faker.text(max_nb_chars=50) for _ in range(500000)]

# Generate 500,000 random currency codes from the provided list
currency_codes = ['AUD', 'BGN', 'BRL', 'CAD', 'CHF', 'CNY', 'CZK', 'DKK', 'GBP', 'HKD', 'HUF', 'IDR', 'ILS', 'INR', 'ISK']
random_currency_codes = np.random.choice(currency_codes, size=500000)

# Generate 500,000 random account numbers between 00100 and 99999
random_account_numbers = np.random.randint(100, 100000, size=500000)

# Create a DataFrame with the specified column order
df = pd.DataFrame({
    'Date': random_dates,
    'Text': random_texts,
    'Account_Number': random_account_numbers,
    'Amount': random_figures,
    'CurrencyCode': random_currency_codes,
})

print("Shape of the DataFrame:", df.shape)
print("Head of the DataFrame:")
print(df.head())

# Save the DataFrame to a CSV file
df.to_csv('Accounting_Data.csv', index=False)
spark.createDataFrame(df).write.mode("overwrite").option("header", True).csv(f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/Files/Accounting_Data.csv")


StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 10, Finished, Available, Finished)

Shape of the DataFrame: (500000, 5)
Head of the DataFrame:
         Date                                     Text  Account_Number  \
0  2023-04-08  Group number family fly onto sometimes.           70394   
1  2023-04-05         Only look research evening tend.           23151   
2  2023-06-29    Should across local treatment effort.           93252   
3  2023-02-03                   To morning respond PM.           81384   
4  2023-05-23      Lead drive line none bring ability.           42217   

      Amount CurrencyCode  
0   13652.04          CZK  
1   16700.00          CAD  
2  125741.71          BRL  
3    9560.13          BRL  
4   45634.67          GBP  


In [9]:
# Create a Spark session
spark = SparkSession.builder.getOrCreate()

# Convert Pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Define OneLake path
output_path = "Files/Accounting_Data.csv"

# Save as CSV to OneLake (Delta Table format is recommended, but CSV works)
spark_df.write.mode("overwrite").option("header", True).csv(f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/{output_path}")

print("CSV saved to OneLake successfully.")


StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 11, Finished, Available, Finished)

CSV saved to OneLake successfully.


In [10]:
print(spark_df.head())

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 12, Finished, Available, Finished)

Row(Date=datetime.date(2023, 4, 8), Text='Group number family fly onto sometimes.', Account_Number=70394, Amount=13652.04, CurrencyCode='CZK')


# Exchange rate calculation

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, col, to_date, round as spark_round
from pyspark.sql.functions import regexp_replace

# Create Spark session
spark = SparkSession.builder.getOrCreate()

# Define Lakehouse paths
output_path1 = "Files/Accounting_Data.csv"
lakehouse_path1 = f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/{output_path1}"
output_path2 = "Files/output_result.csv"
lakehouse_path2 = f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/{output_path2}"

# Read CSVs as Spark DataFrames
df = spark.read.option("header", True).csv(lakehouse_path1)
exchange_rates = spark.read.option("header", True).csv(lakehouse_path2)

# Convert 'Date' columns to just yyyy-MM-dd format before join
df = df.withColumn("Date", to_date(date_format(col("Date"), "yyyy-MM-dd"), "yyyy-MM-dd"))
exchange_rates = exchange_rates.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

print(df.head(5))
print(exchange_rates.head(5))

# Join on 'CurrencyCode' and 'Date'
merged_df = df.join(exchange_rates, on=["CurrencyCode", "Date"], how="left")

# Convert 'Amount' and 'ExchangeRate' to float
merged_df = merged_df.withColumn("Amount", col("Amount").cast("double"))
merged_df = merged_df.withColumn("ExchangeRate", col("ExchangeRate").cast("double"))

# Calculate EUR_Amount and round to 2 decimal places
merged_df = merged_df.withColumn("EUR_Amount", spark_round(col("Amount") / col("ExchangeRate"), 2))

final_df = merged_df.select(
    col("Date"),
    col("Text"),
    col("Account_Number"),
    col("Amount").cast("float"),
    col("CurrencyCode"),
    col("EUR_Amount").cast("float"),
    col("ExchangeRate").cast("float")
)

# Save as CSV to OneLake
output_path = "Files/Accounting_Data_with_EUR_final.csv"
final_df.write.mode("overwrite").option("header", True).csv(
    f"abfss://Fabric_Test_Workspace@onelake.dfs.fabric.microsoft.com/Fabric_Test.Lakehouse/{output_path}"
)

print("DataFrame with EUR amount rounded to 2 decimal places saved.")

print(final_df.head(5))



StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 13, Finished, Available, Finished)

[Row(Date=datetime.date(2023, 1, 23), Text='Gas add how government new morning human.', Account_Number='96471', Amount='35888.52', CurrencyCode='BRL'), Row(Date=datetime.date(2023, 3, 17), Text='Rich on leave my foot network base produce.', Account_Number='34461', Amount='32150.15', CurrencyCode='CAD'), Row(Date=datetime.date(2023, 6, 20), Text='Picture some thousand like action.', Account_Number='5791', Amount='118769.44', CurrencyCode='CZK'), Row(Date=datetime.date(2023, 4, 1), Text='Turn large land because.', Account_Number='65575', Amount='127079.06', CurrencyCode='IDR'), Row(Date=datetime.date(2023, 5, 10), Text='Thus parent data simple ball see worker.', Account_Number='84433', Amount='96100.27', CurrencyCode='CHF')]
[Row(Date=datetime.date(2023, 2, 15), CurrencyCode='ILS', ExchangeRate='3.773'), Row(Date=datetime.date(2023, 2, 15), CurrencyCode='IDR', ExchangeRate='16320.55'), Row(Date=datetime.date(2023, 2, 15), CurrencyCode='HUF', ExchangeRate='380.58'), Row(Date=datetime.date

In [12]:
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")

StatementMeta(, 5662a9ae-905d-416d-ab35-0a5a7378872a, 14, Finished, Available, Finished)

Execution time: 69.72 seconds
